# Steady flow inside an elbow (2D)
This notebook implements a steady incompressible Navier-Stokes solver for the flow inside an elbow.

The problem is strong form reads:
\begin{equation}
\left\{
\begin{array}{ll}
    \nabla \cdot \mathbf{u} =0& in\;\Omega\\
    \displaystyle\left(\mathbf{u}\cdot \nabla\right)\mathbf{u}= \frac{1}{Re}\Delta \mathbf{u}-\nabla p & in\;\Omega\\ & \\
    \mathbf{u} = \mathbf{i} & on\;\Gamma_{in}\\
    \mathbf{u} = \mathbf{0} & on\;\Gamma_w\\
    \left(\frac{1}{Re}\nabla \mathbf{u}-p\mathbb{I}\right)\cdot \mathbf{n}=\mathbf{0} & on \;\Gamma_{out}
\end{array}
\right.
\end{equation}

In [7]:
import tqdm
import numpy as np

# Mesh generation
import dolfinx
from mpi4py import MPI
from dolfinx import mesh
from dolfinx.io import gmshio, XDMFFile
from dolfinx import fem
from dolfinx.fem import Function, FunctionSpace, dirichletbc, locate_dofs_topological, form, assemble_scalar
import ufl
from ufl import grad, div, nabla_grad, dx, inner, dot
from petsc4py import PETSc

import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import cm

plt.rcParams.update({
  "text.usetex": True,
  "font.family": "serif"
})

rcParams['text.latex.preamble'] = r'\usepackage{amssymb} \usepackage{amsmath} \usepackage{amsthm} \usepackage{mathtools}'

The first steps consist in defining the geometry, the functional spaces and the boundary conditions.

In [8]:
gdim = 2

domain, ct, ft = gmshio.read_from_msh("elbow.msh", MPI.COMM_WORLD, gdim = gdim)

fdim = gdim - 1

inl_marker  = 10
walls_marker = 20
out_marker  = 30

P2 = ufl.VectorElement("Lagrange", domain.ufl_cell(), 2)
P1 = ufl.FiniteElement("Lagrange", domain.ufl_cell(), 1)
V, Q = FunctionSpace(domain, P2), FunctionSpace(domain, P1)

# Define the BC
u_in = 1.
def inlet_velocity_expression(x):
    values = np.zeros((gdim, x.shape[1]),dtype=PETSc.ScalarType)
    values[0] = 1. + 0.0 * x[1]
    return values
in_velocity = Function(V)
in_velocity.interpolate(inlet_velocity_expression)
bc_in = dirichletbc(in_velocity, locate_dofs_topological(V, fdim, ft.find(inl_marker)))

## Walls: no slip
noslip = np.zeros(domain.geometry.dim, dtype=PETSc.ScalarType)
bc_w = dirichletbc(noslip, locate_dofs_topological(V, fdim, ft.find(walls_marker)), V)

bcs = [bc_in, bc_w]

Info    : Reading 'elbow.msh'...
Info    : 23 entities
Info    : 4702 nodes
Info    : 9402 elements
Info    : Done reading 'elbow.msh'


Let us define the functional spaces
\begin{equation}
\begin{split}
\mathcal{V} &= \left\{\mathbf{v}\in\left[\mathcal{H}^1(\Omega)\right]^2:\,
                                                                        \left.\mathbf{v}\right|_{\Gamma_{in}} = \mathbf{u}_{in}\;,\;
                                                                        \left.\mathbf{v}\right|_{\Gamma_w} = \mathbf{0}\right\}\\
\mathcal{V}_0 &= \left\{\mathbf{v}\in\left[\mathcal{H}^1(\Omega)\right]^2:\,
                                                                        \left.\mathbf{v}\right|_{\Gamma_{in}\cup\Gamma_w} = \mathbf{0}\right\}\\
\mathcal{Q} &= \left\{q\in\mathcal{L}^2(\Omega)\right\}
\end{split}
\end{equation}

The steady Navier-Stokes equations are non-linear thus an iterative method is required. In this notebook a fixed point iteration is used: so that, the weak formulation reads
\begin{equation}
\int_\Omega\left(\mathbf{u}^k\cdot\nabla\right)\mathbf{u}\,d\Omega+\frac{1}{Re}\int_\Omega\nabla\mathbf{u}\cdot\mathbf{v}\,d\Omega -\int_\Omega p \,\nabla\cdot \mathbf{v}\,d\Omega +
\int_\Omega q \,\nabla\cdot \mathbf{u} = 0\qquad \forall \mathbf{v}\in\mathcal{V}_0, q\in\mathcal{Q}
\end{equation}
which results in the following saddle point system
\begin{equation}
\left[
\begin{array}{ll}
C(U^{k})+A & B^T \\
B & 0
\end{array}
\right]\cdot
\left[
\begin{array}{l}
\mathbf{U} \\
\mathbf{P}
\end{array}
\right] = 
\left[
\begin{array}{l}
\mathbf{F} \\
\mathbf{0}
\end{array}
\right]
\end{equation}

In [9]:
(u, p) = ufl.TrialFunction(V), ufl.TrialFunction(Q)
(v, q) = ufl.TestFunction(V), ufl.TestFunction(Q)

uOld   = Function(V)
uTilde = Function(V)

f = fem.Constant(domain, (PETSc.ScalarType(0), PETSc.ScalarType(0)))

Re = 100
nu = fem.Constant(domain, PETSc.ScalarType(1. / Re))

a = form([[ (inner( dot(uOld, nabla_grad(u)) ,v) + nu * inner(grad(u), grad(v))) * dx, inner(p, div(v)) * dx],
          [inner(div(u), q) * dx, None]])
L = form([inner(f, v) * dx, inner(fem.Constant(domain, PETSc.ScalarType(0)), q) * dx])

# we will use a block-diagonal preconditioner to solve this problem
a_p = [[form(inner(grad(u), grad(v)) * dx), None],
       [None, form(inner(p, q) * dx)]]

Let's assemble the linear system to solve at each iteration using a block iterative solver.

In [10]:
A = fem.petsc.create_matrix_block(a)
P = fem.petsc.assemble_matrix_block(a_p, bcs=bcs)
P.assemble()
b = fem.petsc.assemble_vector_block(L, a, bcs=bcs)

# Build IndexSets for each field (global dof indices for each field)
V_map = V.dofmap.index_map
Q_map = Q.dofmap.index_map
offset_u = V_map.local_range[0] * V.dofmap.index_map_bs + Q_map.local_range[0]
offset_p = offset_u + V_map.size_local * V.dofmap.index_map_bs
is_u = PETSc.IS().createStride(V_map.size_local * V.dofmap.index_map_bs, offset_u, 1, comm=PETSc.COMM_SELF)
is_p = PETSc.IS().createStride(Q_map.size_local, offset_p, 1, comm=PETSc.COMM_SELF)

# Create LU solver
ksp = PETSc.KSP().create(domain.comm)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("superlu_dist")

# Create Krylov solver (iterative MINRES solver is used)
# ksp = PETSc.KSP().create(domain.comm)
# ksp.setOperators(A, P)
# ksp.setTolerances(atol=1e-6, rtol=1e-6, max_it = 4000)
# ksp.setType("minres")
# ksp.getPC().setType("fieldsplit")
# ksp.getPC().setFieldSplitType(PETSc.PC.CompositeType.ADDITIVE)
# ksp.getPC().setFieldSplitIS(
#     ("u", is_u),
#     ("p", is_p))

# # Configure velocity and pressure sub KSPs
# ksp_u, ksp_p = ksp.getPC().getFieldSplitSubKSP()
# ksp_u.setType("preonly")
# ksp_u.getPC().setType("gamg")
# ksp_p.setType("preonly")
# ksp_p.getPC().setType("jacobi")

# Monitor the convergence of the KSP
# opts = PETSc.Options()
# opts["ksp_monitor"] = None
# opts["ksp_view"] = None
# ksp.setFromOptions()

Then, an auxiliary variational problem is defined to update the velocity
\begin{equation}
\int_\Omega \mathbf{u}^{k+1} \cdot \mathbf{v} \,d\Omega= \int_\Omega\left( \alpha\tilde{\mathbf{u}}+(1-\alpha)\mathbf{u}^{k}\right) \cdot \mathbf{v} \,d\Omega
\end{equation}
given $\tilde{\mathbf{u}}$ the solution of the linear system and $\alpha$ as the under-relaxation constant.

In [11]:
alpha = 0.8

update_a = form(inner(u, v) * dx)
update_L = form(inner(alpha * uTilde + (1.-alpha) * uOld, v) * dx)

update_A = fem.petsc.assemble_matrix(update_a)
update_A.assemble()
update_b = fem.petsc.create_vector(update_L)

solverUp = PETSc.KSP().create(domain.comm)
solverUp.setOperators(update_A)
solverUp.setType(PETSc.KSP.Type.CG)
solverUp.getPC().setType(PETSc.PC.Type.SOR)

An iterative method is proposed whose stop criterion is the absolute difference between the updated field and the old one
\begin{equation}
\|\mathbf{u}^{k+1}-\mathbf{u}^{k}\|_{L^2} < tol
\end{equation}

In [12]:
u_sol, p_sol = Function(V), Function(Q)

iter = 1
error = 1.
tol = 1e-3
maxIter = 50

R = 1

Re_computed = u_in * R * Re

print(' ')
print('The Reynolds number is: {:.0f}'.format(Re_computed))
print('The relaxation factor is {:.2f}'.format(alpha)+' and the tolerance for the loop is {:.2e}'.format(tol))
print(' ')

u_xdmf = XDMFFile(domain.comm, "U_withRe_"+str(Re_computed)+".xdmf", "w")
u_xdmf.write_mesh(domain)
u_xdmf.write_function(u_sol, iter)

while error > tol:
    A.zeroEntries()
    fem.petsc.assemble_matrix_block(A, a, bcs=bcs)
    A.assemble()

    # Compute uTilde
    x = A.createVecRight()
    ksp.solve(b, x)
    offset = V_map.size_local * V.dofmap.index_map_bs
    uTilde.x.array[:offset] = x.array_r[:offset]
    p_sol.x.array[:(len(x.array_r) - offset)] = x.array_r[offset:]

    # Update velocity
    with update_b.localForm() as loc:
        loc.set(0)
    fem.petsc.assemble_vector(update_b, update_L)
    update_b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    solverUp.solve(update_b, u_sol.vector)
    u_sol.x.scatter_forward()

    # Compute error
    error = np.sqrt(assemble_scalar(form( inner(u_sol - uOld, u_sol - uOld) * dx)))
    print(f'Iter {iter+0:02} | Error: {float(error):.3e}')
    iter += 1
    u_xdmf.write_function(u_sol, iter)

    with u_sol.vector.localForm() as loc_, uOld.vector.localForm() as loc_n:
        loc_.copy(loc_n)

    if iter >= maxIter:
        error = 0
        print(' ')
        print('Warning: max iteration reached!')

u_xdmf.close()

 
The Reynolds number is: 100
The relaxation factor is 0.80 and the tolerance for the loop is 1.00e-03
 
Iter 01 | Error: 1.858e+00
Iter 02 | Error: 4.671e-01
Iter 03 | Error: 2.139e-01
Iter 04 | Error: 1.455e-01
Iter 05 | Error: 9.642e-02
Iter 06 | Error: 5.567e-02
Iter 07 | Error: 2.783e-02
Iter 08 | Error: 1.247e-02
Iter 09 | Error: 5.398e-03
Iter 10 | Error: 1.118e-02
Iter 11 | Error: 8.011e-03
Iter 12 | Error: 2.706e-03
Iter 13 | Error: 1.055e-03
Iter 14 | Error: 3.879e-04
